In [33]:
from ase.io import read, write
from ase.visualize import view
from ase.build.molecule import molecule
from ase.io import write
from ase.io.pov import get_bondpairs, set_high_bondorder_pairs
import matplotlib.pyplot as plt
from ase.visualize.plot import plot_atoms
import numpy as np
from ase import Atoms

# distance in Angstrom between the given chalcogen and a hydrogen
chalcogen_hydrogen_distances = {'Se': 1.46, 'S': 1.3, 'Te': 1.68}

## Functions

In [90]:
def get_scaled_vector(base_point, vector, scale):
    """
    Plot a point at the end of a scaled vector in 3D space.

    Parameters:
    - base_point: The starting point of the vector (a tuple or list of x, y, z).
    - vector: The direction of the vector (a tuple or list of x, y, z).
    - scale: The scaling factor for the vector length.
    """
    # Convert base_point and vector to NumPy arrays
    base_point = np.array(base_point)

    #normalize the vector
    vector = np.array(vector)
    unit_vector = vector / np.linalg.norm(vector)
    
    # Scale the vector
    scaled_vector = scale * unit_vector
    
    # Calculate the end point of the scaled vector
    end_point = base_point + scaled_vector

    return end_point

import numpy as np

import numpy as np

def vector_from_points(point1, point2):
    """
    Returns the vector from point1 to point2.
    
    Parameters:
    point1 (array-like): Coordinates of the first point.
    point2 (array-like): Coordinates of the second point.
    
    Returns:
    numpy.ndarray: The vector from point1 to point2.
    """
    # Convert points to numpy arrays
    point1 = np.array(point1)
    point2 = np.array(point2)
    
    # Calculate the vector from point1 to point2
    vector = point2 - point1
    
    return vector

def get_neighbors(vectors, cutoff):
    return np.array([v for v in vectors if np.linalg.norm(v) <= cutoff])




## Trying again but this time with average vector

In [140]:
def add_hydrogens(inorganic, filename, plot_structure=True): 
    visited, chalc_pos, hydrogen_pos , vectors = [], [], [], []
    chalcogen = ''

    cutoff = min(inorganic.get_cell_lengths_and_angles()[:4])
    print(cutoff)

    # collecting the positions of the chalcogens
    for atom, pos in zip(inorganic.get_chemical_symbols(), inorganic.get_positions()):
        if str(atom) in chalcogen_hydrogen_distances.keys():
            chalc_pos.append(pos)
            chalcogen = str(atom)

    for point in chalc_pos: # going through chalcogens
        print(point)
        if point.tolist() not in visited: 
            vectors = [vector_from_points(point, p) for p in inorganic.get_positions()]

            # get a vector from the visited chalcogen to the neighbors within a certain cutoff
            neighbor_vectors = get_neighbors(vectors, cutoff) 

            # Calculate the average vector
            average_vector = np.mean(neighbor_vectors, axis=0)

            # get (-) vector which is the furthest from all other points
            flipped_vector = -1*average_vector

            # plot a hydrogen at the end of the vector starting from the chalcogen
            end_point = get_scaled_vector(point, flipped_vector, chalcogen_hydrogen_distances[chalcogen])
            hydrogen_pos.append(end_point)

            visited.append(point.tolist()) # mark as visited

    # using positions to attach hydrogens to inorganic
    for h in hydrogen_pos:
        hydrogen_atom = Atoms('H', positions=[h]) 
        inorganic.extend(hydrogen_atom)

    if plot_structure:
        fig, ax = plt.subplots(1,3, figsize=(12,8))
        plot_atoms(inorganic, ax[0], radii=0.5, rotation=('-90x,0y,0z'))
        plot_atoms(inorganic, ax[1], radii=0.5, rotation=('0x,90y,0z'))
        plot_atoms(inorganic, ax[2], radii=0.5, rotation=('0x,0y,90z'))
    write(f"{path}{filename}_FINAL.vasp", inorganic)
    print(f"Written to {path}{filename}_FINAL.vasp")

path = "/Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/airss_practice/"
filename = "inorganic2"
inorganic = read(f'{path}{filename}.vasp')
print(inorganic.get_cell_lengths_and_angles())
add_hydrogens(inorganic, filename, plot_structure=False)

[ 7.28999996  5.87900019 28.0720005  90.         93.80000305 90.        ]
5.8790001869
[3.67453693 1.15816304 1.20164115]
[ 2.74431564  4.72083697 15.2067836 ]
[0.02953694 4.09766331 1.20164115]
[-0.90068434  1.78133706 15.2067836 ]
[ 1.80493429  3.99772019 26.82544852]
[ 2.73515552  1.88128    12.82030691]
[ 5.44993406  1.05822009 26.82544852]
[ 6.38015528  4.82078009 12.82030691]
Written to /Users/adrianaladera/Desktop/MIT/research/mochas/ALL_STRUCTURES/airss_practice/inorganic2_FINAL.vasp


/Users/adrianaladera/opt/anaconda3/envs/deeznuts/lib/python3.11/site-packages/ase/utils/__init__.py:62: FutureWarning: Please use atoms.cell.cellpar() instead
  warnings.warn(warning)
